In [24]:
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer
from keras.layers import Input, Dense
import pandas as pd

In [11]:
train = pd.read_csv("train.csv").dropna()
test = pd.read_csv("test.csv").dropna()

def append_sentiment(df):
    return df.sentiment.str.strip() + " " + df.text.str.strip()

train["text"] = append_sentiment(train)
train["selected_from"] = train.apply(lambda x: x.text.find(x.selected_text), axis=1)
train["selected_to"] = train.apply(lambda x: x.selected_from + len(x.selected_text), axis=1)
x_train = train.text
y_train = train[["selected_from", "selected_to"]]
x_train, x_validation, y_train, y_validation = train_test_split(x_train, y_train, test_size=0.15)

In [27]:
tokenizer = BertTokenizer.from_pretrained("bert-base-cased")
embedding = tokenizer(x_train.to_list(), padding="max_length", max_length=256, truncation=True)

256